In [ ]:
# connecting to blockchain
from web3 import Web3
from web3.middleware import geth_poa_middleware

w3 = Web3(Web3.HTTPProvider('https://rinkeby.infura.io/v3/<id>'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

print(w3.isConnected())

In [ ]:
def get_guess():
    current_hash = str(w3.eth.get_block('latest')["hash"].hex())
    print(current_hash)
    current_hash = current_hash[2:3]
    if current_hash.lower() in ["8", "9", "a", "b", "c", "d", "e", "f"]:
        return True
    else:
        return False

In [ ]:
contract_address = "0xBD3015Fd34047Ac9f7d5eFE29A3724E30971ed09"
contract_abi = """[
  {
    "inputs": [],
    "stateMutability": "nonpayable",
    "type": "constructor"
  },
  {
    "inputs": [],
    "name": "consecutiveWins",
    "outputs": [
      {
        "internalType": "uint256",
        "name": "",
        "type": "uint256"
      }
    ],
    "stateMutability": "view",
    "type": "function",
    "constant": true,
    "signature": "0xe6f334d7"
  },
  {
    "inputs": [
      {
        "internalType": "bool",
        "name": "_guess",
        "type": "bool"
      }
    ],
    "name": "flip",
    "outputs": [
      {
        "internalType": "bool",
        "name": "",
        "type": "bool"
      }
    ],
    "stateMutability": "nonpayable",
    "type": "function",
    "signature": "0x1d263f67"
  }
]"""

contract = w3.eth.contract(address=contract_address, abi=contract_abi)

address = "player"
sec_key = "player secret key"

guess = get_guess()
transaction = contract.functions.flip(guess).buildTransaction()
transaction.update({ 'gas' : 150000 })
transaction.update({'nonce': w3.eth.get_transaction_count(address)})
signed_tx = w3.eth.account.sign_transaction(transaction, sec_key)

txn_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
txn_receipt = w3.eth.wait_for_transaction_receipt(txn_hash)
print(txn_receipt)

In [ ]:
print(contract.functions.consecutiveWins().call())